<h2 style="color: #004C47;">Installation des bibliothèques nécessaires </h2>
<div style="border: 2px solid #01796F; background-color: #E0F2F1; color: #01796F; padding: 15px; border-radius: 5px; font-size: 16px;">
    <strong>
    
**pip install beautifulsoup4**

**pip install requests**

**pip install PyPDF2**

**pip install pdfplumber**   </div>


In [1]:
# Imports internes
from io import BytesIO
import logging
import os
import PyPDF2
import re
import sqlite3
import time
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime


# Imports externes
import pdfplumber
import requests
from bs4 import BeautifulSoup

<h2 style="color: #004C47;">Chargement et rechargement du module tools</h2> 

In [21]:
%load_ext autoreload
%autoreload 2

import tools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<h2 style="color: #004C47;">1. Scraper les articles bioRxiv liés à la bioinformatique</h2>


<div style="border: 2px solid #01796F; background-color: #E0F2F1; color: #01796F; padding: 15px; border-radius: 5px; font-size: 16px;">
    <strong>Objectif:</strong> Ce script permet d'archiver automatiquement les dépôts logiciels non encore archivés dans la base Software Heritage.Récupérer les articles publiés sur bioRxiv dans le domaine de la bioinformatique, en collectant les informations essentielles pour une analyse et un traitement ultérieurs.

Informations collectées :

-Titre de l’article : Titre complet décrivant le sujet de recherche.

-Résumé (abstract) : Vue d’ensemble concise du contenu scientifique.

-Auteurs : Liste des auteurs, ordonnée telle qu’elle apparaît sur bioRxiv.

-Date de publication : Date de mise en ligne pour le suivi des recherches.

-DOI : Identifiant unique facilitant la référence et l’accès.

-URL de l’article : Lien direct vers la page de l’article sur bioRxiv.

-Mots-clés : Mots-clés associés pour décrire le domaine ou sujet de recherche.

-URLs des dépôts logiciels : Liens vers les dépôts de code (GitHub, GitLab) associés aux ressources logicielles.

 
</div>

In [33]:
import tools

def main():
    """
    Scrape les articles bioRxiv dans le domaine de la bioinformatique
    et enregistre les informations essentielles dans la base de données.
    """
    tools.scrape_articles(total_pages=10)

if __name__ == "__main__":
    main()

2024-12-06 16:20:52,069 - INFO - Base de données créée et initialisée avec succès.
2024-12-06 16:20:52,070 - INFO - Scraping page 1 sur 10
2024-12-06 16:20:53,767 - INFO - L'article avec DOI 10.1101/2024.12.02.625958 existe déjà dans la base.
2024-12-06 16:20:53,768 - INFO - Connexion à la base de données fermée.


<h2 style="color: #004C47;">2. Extraction des URLs des Dépôts Logiciels</h2>


<div style="border: 2px solid #01796F; background-color: #E0F2F1; color: #01796F; padding: 15px; border-radius: 5px; font-size: 16px;">
    <strong>Objectif : </strong> Extraire automatiquement les liens vers les dépôts logiciels (GitHub, GitLab) mentionnés dans les articles bioRxiv liés à la bioinformatique. Ce script parcourt les PDF et les résumés (abstracts) des articles pour identifier et collecter ces URLs de dépôt, qui seront ensuite utilisés pour archivage ou analyse.

 </div>


In [34]:

import tools
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def main():
    """
    Traite les articles pour extraire les liens des dépôts logiciels
    (GitHub, GitLab) et les associe aux articles dans la base de données.
    """
    tools.extract_repository_links()

if __name__ == "__main__":
    main()

2024-12-06 16:21:00,992 - INFO - Base de données créée et initialisée avec succès.
2024-12-06 16:21:00,994 - INFO - Connexion à la base de données fermée.


<h2 style="color: #004C47;">3. Archivage via Software Heritage</h2>

<div style="border: 2px solid #01796F; background-color: #E0F2F1; color: #01796F; padding: 15px; border-radius: 5px; font-size: 16px;">
    <strong>Objectif : </strong>  Assurer l’archivage à long terme des dépôts logiciels associés aux articles via l’API de Software Heritage.

-Vérification : Pour chaque dépôt, le script vérifie s’il est déjà archivé.

-Soumission automatique : Si non archivé, une demande d’archivage est soumise automatiquement.

-Mise à jour : La base de données est mise à jour avec les liens et dates d’archivage.

 </div>

In [35]:
import logging
from tools import archive_repositories

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def main():
    """
    Archive les dépôts présents dans la base de données.
    """
    archive_repositories(db_name="bioinformatics_article.db")

if __name__ == "__main__":
    main()

2024-12-06 16:21:05,409 - INFO - Base de données créée et initialisée avec succès.
2024-12-06 16:21:05,410 - INFO - Nombre de dépôts à traiter : 0
2024-12-06 16:21:05,411 - INFO - Aucun dépôt à traiter. Fin du script.
2024-12-06 16:21:05,412 - INFO - Connexion à la base de données fermée.


<h2 style="color: #004C47;">4. Re-vérification de l’archivage pour les dépôts soumis</h2>

<div style="border: 2px solid #01796F; background-color: #E0F2F1; color: #01796F; padding: 15px; border-radius: 5px; font-size: 16px;">
    <strong>Objectif : </strong>  Vérifier l’état d’archivage des dépôts soumis pour archivage auprès de Software Heritage.

-Vérification périodique : Ce script revisite les dépôts soumis (ceux en attente d’archivage) pour voir si l’archivage a été complété.

-Mise à jour : Si un dépôt est archivé avec succès, le script enregistre le lien et la date d’archivage dans la base de données.
</div>


In [37]:
import tools
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def main():
    """
    Re-vérifie les dépôts marqués comme en cours d'archivage dans Software Heritage.
    Met à jour la base de données avec les résultats.
    """
    tools.recheck_archived_repositories()

if __name__ == "__main__":
    main()

2024-12-06 16:21:12,716 - INFO - Base de données créée et initialisée avec succès.
2024-12-06 16:21:12,717 - INFO - Nombre de dépôts à re-vérifier : 1
2024-12-06 16:21:12,717 - INFO - Re-vérification des dépôts marqués comme en cours d'archivage.
2024-12-06 16:21:12,718 - INFO - Connexion à la base de données fermée.
2024-12-06 16:21:12,721 - INFO - Base de données créée et initialisée avec succès.
2024-12-06 16:21:12,721 - INFO - Re-vérification de l'archivage pour le dépôt : https://github.com/royal-dargon/deepGPS/.
2024-12-06 16:21:12,795 - ERROR - L'URL https://github.com/royal-dargon/deepGPS n'est pas trouvée dans l'API Software Heritage.
2024-12-06 16:21:12,796 - INFO - Connexion à la base de données fermée.


<h2 style="color: #004C47;">Affichage des tables et colonnes de la base de données </h2>

<div style="border: 2px solid #01796F; background-color: #E0F2F1; color: #01796F; padding: 15px; border-radius: 5px; font-size: 16px;">
    <strong>Objectif : </strong> Visualiser la structure de la base de données en restant sur le fichier jupyter pour confirmer la présence des tables attendues et des colonnes associées.
</div>

In [38]:
import sqlite3
import pandas as pd

def show_tables(db_name="bioinformatics_article.db"):
    """
    Affiche le contenu des tables articles, code_repositories et articles_code_repositories
    dans la base de données SQLite avec les noms des colonnes.
    """
    # Connexion à la base de données
    conn = sqlite3.connect(db_name)

    # Fonction pour afficher le contenu d'une table sous forme de DataFrame
    def show_table_content(table_name):
        print(f"\nContenu de la table '{table_name}':")
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql_query(query, conn)
        display(df)  # Affiche le DataFrame dans le notebook

    # Afficher le contenu des tables
    show_table_content("articles")
    show_table_content("code_repositories")
    show_table_content("articles_code_repositories")

    # Fermer la connexion
    conn.close()

# Exécuter la fonction
show_tables()


Contenu de la table 'articles':


,id,title,link,doi,date,pdf_link,abstract,is_article_processed,contains_valid_repo_link
0,23,BC-Design: A Biochemistry-Aware Framework for ...,https://www.biorxiv.org/content/10.1101/2024.1...,10.1101/2024.10.28.620755,2024-01-01,https://www.biorxiv.org/content/10.1101/2024.1...,"Inverse protein folding, which aims to design ...",1,0
1,24,Parallel maximal common subgraphs with labels ...,https://www.biorxiv.org/content/10.1101/2024.0...,10.1101/2024.05.10.593525,2024-01-01,https://www.biorxiv.org/content/10.1101/2024.0...,Advances in graph algorithmics have allowed in...,1,0
2,25,GenomicSign: A computational method to discove...,https://www.biorxiv.org/content/10.1101/2024.1...,10.1101/2024.11.05.622192,2024-01-01,https://www.biorxiv.org/content/10.1101/2024.1...,Molecular diagnostics for the rapid identifica...,1,0
3,26,"Vcfexpress: flexible, rapid user-expressions t...",https://www.biorxiv.org/content/10.1101/2024.1...,10.1101/2024.11.05.622129,2024-01-01,https://www.biorxiv.org/content/10.1101/2024.1...,Motivation: Variant Call Format (VCF) files ar...,1,1
4,27,pLM-DBPs: Enhanced DNA-Binding Protein Predict...,https://www.biorxiv.org/content/10.1101/2024.1...,10.1101/2024.10.04.616755,2024-01-01,https://www.biorxiv.org/content/10.1101/2024.1...,DNA-binding proteins (DBPs) play critical role...,1,0
...,...,...,...,...,...,...,...,...,...
110,133,Joint analysis of RNA-DNA and DNA-DNA interact...,https://www.biorxiv.org/content/10.1101/2024.1...,10.1101/2024.11.30.626180,2024-01-01,https://www.biorxiv.org/content/10.1101/2024.1...,"At the moment, many non-coding RNAs that perfo...",1,0
111,134,Scalable Guide Tree Construction Using Quantum...,https://www.biorxiv.org/content/10.1101/2024.1...,10.1101/2024.11.30.626202,2024-01-01,https://www.biorxiv.org/content/10.1101/2024.1...,A guide tree directs the sequence alignment or...,1,0
112,135,Automated Cell Type Annotation with Reference ...,https://www.biorxiv.org/content/10.1101/2024.1...,10.1101/2024.11.30.626130,2024-01-01,https://www.biorxiv.org/content/10.1101/2024.1...,RNA sequencing (scRNA-seq) technologies have r...,1,0
113,136,Predicting Tissue of Origin from Bulk Tumor Ge...,https://www.biorxiv.org/content/10.1101/2024.1...,10.1101/2024.12.01.626105,2024-01-01,https://www.biorxiv.org/content/10.1101/2024.1...,Identifying the tissue of origin for cancers i...,1,0



Contenu de la table 'code_repositories':


,code_repo_id,code_repo_url,is_archived_in_swh,swh_archive_link,swh_date_last_archive
0,1,https://github.com/brentp/vcfexpress,1,https://archive.softwareheritage.org/browse/or...,2024-11-14T12:33:17.227000+00:00
1,2,https://github.com/satijalab/seurat,1,https://archive.softwareheritage.org/browse/or...,2024-11-14T12:33:15.176000+00:00
2,3,https://github.com/RenanSimoesBR,1,https://archive.softwareheritage.org/browse/or...,2024-11-14T12:33:15.696000+00:00
3,4,https://github.com/QuackenbushLab/tcga-data-nf,1,https://archive.softwareheritage.org/browse/or...,2024-11-14T12:33:16.465000+00:00
4,5,https://github.com/QuackenbushLab/NetworkDataC...,1,https://archive.softwareheritage.org/browse/or...,2024-11-14T12:33:17.982000+00:00
...,...,...,...,...,...
79,80,https://github.com/yukt/aggRSquare.git,1,https://archive.softwareheritage.org/browse/or...,2024-12-06T15:18:10.309000+00:00
80,81,https://github.com/tsenoner/protspace,1,https://archive.softwareheritage.org/browse/or...,2024-12-06T15:18:10.311000+00:00
81,82,https://github.com/genbio-ai/ModelGenerator,1,https://archive.softwareheritage.org/browse/or...,2024-12-06T15:18:10.309000+00:00
82,83,https://github.com/genbio-ai/AIDOandonHuggingFace,1,https://archive.softwareheritage.org/browse/or...,2024-12-06T15:18:10.316000+00:00



Contenu de la table 'articles_code_repositories':


,article_id,code_repo_id
0,26,1
1,28,2
2,28,3
3,33,4
4,33,5
...,...,...
81,131,80
82,130,81
83,132,82
84,132,83
